In [1]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import sys
drive.mount('/mnt/drive')

storage_path = "/mnt/drive/MyDrive/cs224u_final_project"
sys.path.append(storage_path)
from compgen import recogs_exact_match

Mounted at /mnt/drive


In [3]:
import pandas as pd
import plotly.express as px

In [4]:
df_training = pd.read_parquet(f"{storage_path}/gpt4_examples_200_num_tokens.parquet")
df_training.head()

,input,output,category,output_renum,gpt4_guide,result_guide,guide_correct,prompt_training_base,prompt_training_guide,prompt_training_guide_num_token,prompt_training_base_num_token,gpt4_guide_num_token
24552,A cat lended the strawberry on a pedestal to a...,cat ( 50 ) ; * strawberry ( 5 ) ; pedestal ( 4...,in_distribution,cat ( 1 ) ; * strawberry ( 2 ) ; pedestal ( 3 ...,Step 1: Identify nouns\n- cat ( 1 ); *strawber...,cat ( 1 ); *strawberry ( 2 ); pedestal ( 3 ); ...,True,<s>[INST] A cat lended the strawberry on a ped...,<s>[INST] You are given a sentence and must co...,1056,101,259
3254,A boy rolled the balloon beside the basket .,boy ( 5 ) ; * balloon ( 38 ) ; * basket ( 35 )...,in_distribution,boy ( 1 ) ; * balloon ( 2 ) ; * basket ( 31 ) ...,Step 1: Identify nouns\n- boy ( 1 ); *balloon ...,boy ( 1 ); *balloon ( 2 ); *basket ( 31 ); nmo...,True,<s>[INST] A boy rolled the balloon beside the ...,<s>[INST] You are given a sentence and must co...,998,78,206
10756,Emma rolled the cake in a basin .,Emma ( 8 ) ; * cake ( 25 ) ; basin ( 18 ) ; nm...,in_distribution,Emma ( 1 ) ; * cake ( 2 ) ; basin ( 11 ) ; nmo...,Step 1: Identify nouns\n- Emma (1); *cake (2);...,Emma (1); *cake (2); basin (3); roll (4) AND a...,True,<s>[INST] Emma rolled the cake in a basin . [/...,<s>[INST] You are given a sentence and must co...,997,76,205
74585,The donut was brought to a wolf .,* donut ( 10 ) ; wolf ( 1 ) ; bring ( 52 ) AND...,in_distribution,* donut ( 2 ) ; wolf ( 2 ) ; bring ( 3 ) AND t...,Step 1: Identify nouns\n- *donut (1); *wolf (2...,"*donut (1); *wolf (2); bring (3) AND theme (3,...",False,<s>[INST] The donut was brought to a wolf . [/...,<s>[INST] You are given a sentence and must co...,939,56,148
92102,Emma attempted to walk .,"Emma ( 39 ) ; attempt ( 49 ) AND agent ( 49 , ...",in_distribution,"Emma ( 1 ) ; attempt ( 2 ) AND agent ( 2 , 1 )...",Step 1: Identify nouns\n- Emma ( 1 )\n\nStep 2...,"Emma ( 1 ); attempt ( 2 ) AND agent ( 2 , 1 ) ...",True,<s>[INST] Emma attempted to walk . [/INST] Emm...,<s>[INST] You are given a sentence and must co...,1008,60,220


In [18]:
token_counts = {
    "base": {0:0},
    "guide": {0:0}
}
for num in [10,50,100,170]:
  token_count = df_training[df_training.guide_correct].prompt_training_base_num_token.iloc[:num].sum()
  token_counts["base"][num] = token_count
  token_count = df_training[df_training.guide_correct].prompt_training_guide_num_token.iloc[:num].sum()
  token_counts["guide"][num] = token_count
  # print(f"{num} - {token_count}")
token_counts

{'base': {0: 0, 10: 749, 50: 3685, 100: 7515, 170: 12268},
 'guide': {0: 0, 10: 9998, 50: 49388, 100: 98985, 170: 167610}}

In [19]:
df = pd.read_parquet(f"{storage_path}/model_guide_10_5_preds.parquet")
df.head()

,input,output,category,output_renum,prompt_test_base,prompt_test_guide,gen_test_guide,result_guide,guide_correct
9202,A lion squeezed a balloon on the stage .,lion ( 46 ) ; balloon ( 44 ) ; * stage ( 49 ) ...,passive_to_active,lion ( 1 ) ; balloon ( 2 ) ; * stage ( 3 ) ; n...,<s>[INST] A lion squeezed a balloon on the sta...,<s>[INST] You are given a sentence and must co...,</s></s></s></s></s></s></s></s></s></s></s></...,*lion ( 1 ); *balloon ( 2 ); *stage ( 3 ); nmo...,False
20450,Emma squeezed the professor on the hanger .,Emma ( 48 ) ; * professor ( 59 ) ; * hanger ( ...,passive_to_active,Emma ( 1 ) ; * professor ( 2 ) ; * hanger ( 3 ...,<s>[INST] Emma squeezed the professor on the h...,<s>[INST] You are given a sentence and must co...,</s></s></s></s></s></s></s></s></s></s></s></...,Emma (1); *professor (2); *hanger (3); *squeez...,False
14358,The girl squeezed Liam .,* girl ( 43 ) ; Liam ( 56 ) ; squeeze ( 20 ) A...,passive_to_active,* girl ( 1 ) ; Liam ( 2 ) ; squeeze ( 3 ) AND ...,<s>[INST] The girl squeezed Liam . [/INST],<s>[INST] You are given a sentence and must co...,</s></s></s></s></s></s></s></s></s></s></s></...,*girl ( 1 ); *man ( 2 : Liam ) AND man ( 2 : L...,False
12832,The teacher in the cart cooked Emma .,* teacher ( 31 ) ; * cart ( 6 ) ; Emma ( 47 ) ...,obj_pp_to_subj_pp,* teacher ( 1 ) ; * cart ( 2 ) ; Emma ( 3 ) ; ...,<s>[INST] The teacher in the cart cooked Emma ...,<s>[INST] You are given a sentence and must co...,</s></s></s></s></s></s></s></s></s></s></s></...,*teacher ( 1 ); *cart ( 2 ); *teacher\_in\_car...,False
2052,Michael believed that Mason hoped that the con...,Michael ( 15 ) ; Mason ( 37 ) ; * consumer ( 4...,cp_recursion,Michael ( 6 ) ; Mason ( 2 ) ; * consumer ( 3 )...,<s>[INST] Michael believed that Mason hoped th...,<s>[INST] You are given a sentence and must co...,<s><s> [INST] You are given a sentence and mus...,Michael ( 1 ); *consumer ( 3 ); *boy ( 4 ); Ch...,False


In [39]:
data = []
for inf_type in ["base", "guide"]:
  for data_num in [10,50,100,170]:
    epochs = [1,5,10]
    if data_num == 0:
      checkpoints = [0]*3
      # epochs = [0]
    elif data_num == 10:
      checkpoints = [1,5,10]
    elif data_num == 50:
      checkpoints = [6,31,60]
    elif data_num == 100:
      checkpoints = [12,62,120]
    elif data_num == 170:
      checkpoints = [21,105,210]
    else:
      checkpoints = []
      epochs = []
    for epoch, checkpoint in zip(epochs, checkpoints):
      try:
        file_path = f"{storage_path}/model_{inf_type}_{data_num}_{checkpoint}_preds.parquet"
        df = pd.read_parquet(file_path)
        data.append(
            {
                "epoch": epoch,
                "checkpoint": checkpoint,
                "type": inf_type,
                "training_samples": data_num,
                "accuracy": df[f"{inf_type}_correct"].mean(),
                "training_token_count": token_counts[inf_type][data_num],
                "training_epoch_token_count": epoch * token_counts[inf_type][data_num],
            }
        )
        # print(file_path)
      except Exception as e:
        pass
        # print(e)
    try:
        file_path = f"{storage_path}/model_{inf_type}_0_0_preds.parquet"
        df = pd.read_parquet(file_path)
        data.append(
            {
                "epoch": 0,
                "checkpoint": 0,
                "type": inf_type,
                "training_samples": data_num,
                "accuracy": df[f"{inf_type}_correct"].mean(),
                "training_token_count": 0,
                "training_epoch_token_count": 0,
            }
        )
        # print(file_path)
    except Exception as e:
      pass


In [40]:
df = pd.DataFrame(data)
df.head()

,epoch,checkpoint,type,training_samples,accuracy,training_token_count,training_epoch_token_count
0,1,1,base,10,0.00,749,749
1,5,5,base,10,0.00,749,3745
2,10,10,base,10,0.01,749,7490
3,0,0,base,10,0.00,0,0
4,1,6,base,50,0.00,3685,3685


In [41]:
fig = px.line(df.sort_values(by="epoch"), x="epoch", y="accuracy", symbol="type", line_dash="type", color="training_samples", width=600, height=400)
fig.write_image(f"{storage_path}/accuracy_vs_epoch.pdf")
fig.write_image(f"{storage_path}/accuracy_vs_epoch.jpg")
fig.show()

In [42]:
fig = px.line(df.sort_values(by="training_epoch_token_count"), x="training_epoch_token_count", y="accuracy", symbol="type", line_dash="type", color="training_samples",
              labels={"training_epoch_token_count":"total number of fine-tuning tokens"},
              width=600, height=400)
fig.write_image(f"{storage_path}/accuracy_vs_epochtoken.pdf")
fig.write_image(f"{storage_path}/accuracy_vs_epochtoken.jpg")
fig.show()

In [43]:
fig = px.line(df.sort_values(by="training_samples"), x="training_samples", y="accuracy", symbol="type", line_dash="type", color="epoch", width=600, height=400)
fig.write_image(f"{storage_path}/accuracy_vs_data.pdf")
fig.write_image(f"{storage_path}/accuracy_vs_data.jpg")
fig.show()

In [44]:
fig = px.line(df.sort_values(by="training_token_count"), x="training_token_count", y="accuracy", symbol="type", line_dash="type", color="epoch",
              labels={"training_token_count":"total number of tokens in fine-tuning dataset"},
              width=600, height=400)
fig.write_image(f"{storage_path}/accuracy_vs_trainingtoken.pdf")
fig.write_image(f"{storage_path}/accuracy_vs_trainingtoken.jpg")
fig.show()